Use the optimum-cli tool to export a model to the OpenVINO format
optimum-cli export openvino \
    # Specify the model to be exported
    --model "microsoft/Phi-3-mini-4k-instruct" \
    # Define the task for the model, which is text generation with past context
    --task text-generation-with-past \
    # Set the weight format to 4-bit integers for quantization
    --weight-format int4 \
    # Define the group size for quantization
    --group-size 128 \
    # Set the ratio parameter, possibly for optimization
    --ratio 0.6 \
    # Use symmetric quantization
    --sym \
    # Allow execution of remote code if necessary
    --trust-remote-code \
    # Specify the output directory for the exported model
    ./model/phi3-instruct/int4


In [ ]:
! optimum-cli export openvino --model "microsoft/Phi-3-mini-4k-instruct" --task text-generation-with-past --weight-format int4 --group-size 128 --ratio 0.6  --sym  --trust-remote-code ./model/phi3-instruct/int4


Mixed-Precision assignment ---------------------   0% 0/128 • 0:00:00 • -:--:--
Mixed-Precision assignment ---------------------   0% 0/128 • 0:00:00 • -:--:--
Mixed-Precision assignment ---------------------   0% 0/128 • 0:00:00 • -:--:--
Mixed-Precision assignment ---------------------   0% 0/128 • 0:00:00 • -:--:--
Mixed-Precision assignment ---------------------   0% 0/128 • 0:00:00 • -:--:--
Mixed-Precision assignment ---------------------   1% 1/128 • 0:00:00 • -:--:--
Mixed-Precision assignment ---------------------   1% 1/128 • 0:00:00 • -:--:--
Mixed-Precision assignment ---------------------   2% 2/128 • 0:00:00 • 0:00:28
Mixed-Precision assignment ---------------------   2% 2/128 • 0:00:00 • 0:00:28
Mixed-Precision assignment ---------------------   2% 2/128 • 0:00:01 • 0:00:28
Mixed-Precision assignment ---------------------   2% 2/128 • 0:00:01 • 0:00:28
Mixed-Precision assignment ---------------------   2% 2/128 • 0:00:01 • 0:00:28
Mixed-Precision assignment ------------

C:\Users\kinfeylo\AppData\Roaming\Python\Python311\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
Framework not specified. Using pt to export the model.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.

Loading checkpoint shards: 100%|██████████| 2/2 [00:28<00:00, 14.22s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fin

In [2]:
# Import the AutoConfig and AutoTokenizer classes from the transformers library
from transformers import AutoConfig, AutoTokenizer

# Import the OVModelForCausalLM class from the optimum.intel.openvino module
from optimum.intel.openvino import OVModelForCausalLM


C:\Users\kinfeylo\AppData\Roaming\Python\Python311\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [6]:
# Define the directory path where the model is stored
model_dir = './model/phi3-instruct/int4'

In [7]:
# Define a configuration dictionary for the OpenVINO model
ov_config = {
    # Set the performance hint to prioritize low latency
    "PERFORMANCE_HINT": "LATENCY",
    # Specify the number of inference streams to use
    "NUM_STREAMS": "1",
    # Set the directory for caching intermediate data (empty string means no caching)
    "CACHE_DIR": ""
}

In [8]:
ov_model = OVModelForCausalLM.from_pretrained(
    model_dir,
    device='GPU.0',
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
    trust_remote_code=True,
)

Compiling the model to GPU.0 ...


In [11]:
# Load the tokenizer from the specified model directory, allowing remote code execution if necessary
tok = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
# Define a dictionary of keyword arguments for the tokenizer
tokenizer_kwargs = {
    # Specify not to add special tokens to the tokenized output
    "add_special_tokens": False
}

In [13]:
# Define the prompt string with special tokens indicating different roles in the conversation
prompt = "<|system|>You are a helpful AI assistant.<|end|><|user|>can you introduce yourself?<|end|><|assistant|>"

# Tokenize the prompt string, returning the result as PyTorch tensors and applying additional tokenizer settings
input_tokens = tok(prompt, return_tensors="pt", **tokenizer_kwargs)


In [14]:
# Generate a response using the OpenVINO model, with a maximum of 1024 new tokens
answer = ov_model.generate(**input_tokens, max_new_tokens=1024)

In [15]:
# Decode the generated tokens into a string, skipping special tokens, and get the first result
decoded_answer = tok.batch_decode(answer, skip_special_tokens=True)[0]


"You are a helpful AI assistant. can you introduce yourself? Hello! I'm Phi, Microsoft's language model. I'm here to help answer your questions, provide information, and engage in conversation on a wide range of topics. How can I assist you today?"